# Obtener de la API Tweets que no sean retweet y que contengan el hashtag #SpiderMan2 en inglés.
Realizar limpieza en los datos
Evaluar la polaridad

In [8]:
import os
from dotenv import load_dotenv
# Cargar valores del archivo .env en las variables de entorno
load_dotenv()
# Cargar valor del token a variable
bearer_token = os.environ.get("BEARER_TOKEN")

url = "https://api.twitter.com/2/tweets/search/recent"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent":"v2FullArchiveSearchPython"
}
hastag='#SpiderMan2'
params = {
    'query': f'{hastag} -is:retweet lang:en', #Va a traer los que tienen el hashtag '#machinelearning' y el -is:retweet es para que no traigra retweets
    'max_results':100  #Y que traiga 100 resultados maximo
    
}
import requests
response = requests.get(url, headers=headers, params=params)
print(response)
# Generar excepción si la respuesta no es exitosa
if response.status_code != 200:
    raise Exception(response.status_code, response.text)
import pandas as pd
df = pd.json_normalize(response.json()['data'])
df

<Response [200]>


,id,text
0,1454573018534096903,"If true, I just hope we hear some of @dannyelf..."
1,1454271242224250884,Overall #SpiderMan2 is a masterpiece of a movi...
2,1454248405904068613,Spider-Man 2 is trending. You’re welcome. #spi...
3,1454246409935462407,@EthanCastillo05 @YuriLowenthal @insomniacgame...
4,1454230657849245697,Marvel Friday: Fin... (Spider-Man 2) https://t...
...,...,...
89,1452126554574753794,Funko Concept: Spider-Man Armored Suit from 'S...
90,1452121858170576899,@MarvelHDgifs hello how are you today I would ...
91,1452114313112690688,Final Web🕸️Swing ! #SamRaimi's #SpiderMan2 ! h...
92,1452104865656840195,Greatest Inspirational Speech Incoming From Au...


In [9]:
from nltk.tokenize import TweetTokenizer # libreria especifica para los tokenizar tweets 

tt = TweetTokenizer() # intanciamos ya que es un modelo lo que vamos a llamar
# Aplicar Tokenizer a la columna
tokenized_text = df['text'].apply(tt.tokenize)
df["tokenized_text"] = tokenized_text
df=df[['tokenized_text']]
df

,tokenized_text
0,"[If, true, ,, I, just, hope, we, hear, some, o..."
1,"[Overall, #SpiderMan2, is, a, masterpiece, of,..."
2,"[Spider-Man, 2, is, trending, ., You, ’, re, w..."
3,"[@EthanCastillo05, @YuriLowenthal, @insomniacg..."
4,"[Marvel, Friday, :, Fin, ..., (, Spider-Man, 2..."
...,...
89,"[Funko, Concept, :, Spider-Man, Armored, Suit,..."
90,"[@MarvelHDgifs, hello, how, are, you, today, I..."
91,"[Final, Web, 🕸, ️Swing, !, #SamRaimi's, #Spide..."
92,"[Greatest, Inspirational, Speech, Incoming, Fr..."


In [7]:
# Crear lista sin stopwords
import nltk
import re
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def clean(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F" # emoticons
        u"\U0001F300-\U0001F5FF" # symbols & pictographs
        u"\U0001F680-\U0001F6FF" # transport & map symbols
        u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           "]+", flags = re.UNICODE)
    n = regrex_pattern.sub(r'', x)
    if (not x.lower() in stop_words) and (x[0] != '@') and (n != ''):
        return True
    else:
        return False
for index, tweet in enumerate(tokenized_text):
    tweet = [x for x in tweet if clean(x)]
    tokenized_text[index] = tweet

df["tokenized_text"] = tokenized_text
df

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/eduardo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-7-2ebbfde6d3bd>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tokenized_text"] = tokenized_text


,tokenized_text
0,"[true, hope, hear, dannyelfman, theme, Raimi-v..."
1,"[Overall, SpiderMan2, way, delves, Peter, char..."
2,"[Spider-Man, trending, welcome, welcome, #spid..."
3,"[EthanCastillo05, YuriLowenthal, insomniacgame..."
4,"[Marvel, Friday, ..., spidermanhomecoming, spi..."
...,...
91,"[Final, Web, SpiderMan2, #SamRaimi's, #SpiderM..."
92,"[Greatest, Inspirational, Speech, Incoming, Au..."
93,"[Even, Spiderman, views, less, Starfield, stil..."
94,"[Happy, Birthday, one, Sam, Raimi, childhood, ..."


In [4]:
#segunda limpieza
import re
i=0
for text in df['tokenized_text']:
    #text=eval(str_text) # lo convertimos en lista para poder iterar por palabra
    #print("\nTEXTO A ANALIZAR ->",text)
    j=0
    for palabra in text:
        if(re.search('^#+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "#"
            text[j]=palabra
            j=j+1
        elif(re.search('^@+',palabra)):
            palabra = re.sub(r'.', '', palabra, count = 1) # eliminamos el primer caracter que es "@"
            text[j]=palabra
            j=j+1
        elif(re.search('^http+',palabra)):
            text.remove(palabra)  #ACA NO SUMAMOS J PORQUE QUITAMOS UN ELEMENTO DE LA LISTA
        elif(len(palabra)==1):
            text.remove(palabra)  #ACA NO SUMAMOS J PORQUE QUITAMOS UN ELEMENTO DE LA LISTA
        else:
            text[j]=palabra
            j=j+1
    #print("\n TEXTO DEVULETO ->", text) 
    df['tokenized_text'][i] = text
    i=i+1
df

/media/eduardo/Anaconda/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,tokenized_text
0,"[If, true, just, hope, we, hear, some, of, dan..."
1,"[Overall, SpiderMan2, is, of, way, it, delves,..."
2,"[Spider-Man, trending, welcome, You, re, welco..."
3,"[EthanCastillo05, YuriLowenthal, insomniacgame..."
4,"[Marvel, Friday, ..., spidermanhomecoming, spi..."
...,...
91,"[Final, Web, SpiderMan2, #SamRaimi's, #SpiderM..."
92,"[Greatest, Inspirational, Speech, Incoming, Fr..."
93,"[Even, Spiderman, views, is, less, than, Starf..."
94,"[Happy, Birthday, to, the, one, Sam, Raimi, yo..."
